## 37. 序列化二叉树

请实现两个函数，分别用来序列化和反序列化二叉树。

### 分析
通过面试题7:重建二叉树，我们知道可以从前序遍历和中序遍历序列中构造出一棵二叉树。但是这个方法有两个缺点：
1. 此方法要求二叉树中不能有数值重复的节点。
2. 只有当两个序列中所有数据都读出后才能开始反序列化 --> 等待时间很长

实际上，如果在前序遍历的基础上，把遇到的`nullptr`(C++)或者`None`(Python)用特殊符号表示出来，那么在序列化的同时就可以进行反序列化构建二叉树。

[//]: # (<img src="images/img123.png" style="width: 400px;"/>)

In [1]:
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

In [2]:
def serialize(root):
    
    output = []
    if root:
        output.append(root.val)
        output = output + serialize(root.left)
        output = output + serialize(root.right)
    else:
        output.append(None)

    return output


def deserialize(s):
    if len(s) == 0:
        return

    if s[0] is None:
        node = None
    else:
        node = TreeNode(s[0])

    s.pop(0)
    if node:
        node.left = deserialize(s)
        node.right = deserialize(s)

    return node

### Test

In [3]:
root1 = TreeNode(10)
root1.left = TreeNode(6)
root1.right = TreeNode(14)

root1.left.left = TreeNode(4)
root1.left.right = TreeNode(8)

root1.right.left = TreeNode(12)
root1.right.right = TreeNode(16)

print(serialize(root1))
assert serialize(root1) == serialize( deserialize(serialize(root1)) )
print("Test 1 passed.")

##########################################
root2 = TreeNode(1)
root2.left = TreeNode(2)
root2.left.left = TreeNode(4)

root2.right = TreeNode(3)
root2.right.left = TreeNode(5)
root2.right.right = TreeNode(6)

print(serialize(root2))
assert serialize(root2) == serialize( deserialize(serialize(root2)) ) 
print("Test 2 passed.")

##########################################
root3 = TreeNode(5)
root3.left = TreeNode(4)
root3.right = TreeNode(3)
root3.right.left = TreeNode(2)
# root3.right.right.right = TreeNode(1)

print(serialize(root3))
assert serialize(root3) == serialize( deserialize(serialize(root3)) ) 
print("Test 3 passed.")

###########################################
root4 = TreeNode(8)
root4.left = TreeNode(6)
root4.right = TreeNode(7)

root4.left.left = TreeNode(10)
root4.left.right = TreeNode(5)

root4.right.left = TreeNode(9)
root4.right.right = TreeNode(10)

print(serialize(root4))
assert serialize(root4) == serialize( deserialize(serialize(root4)) ) 
print("Test 4 passed.")

[10, 6, 4, None, None, 8, None, None, 14, 12, None, None, 16, None, None]
Test 1 passed.
[1, 2, 4, None, None, None, 3, 5, None, None, 6, None, None]
Test 2 passed.
[5, 4, None, None, 3, 2, None, None, None]
Test 3 passed.
[8, 6, 10, None, None, 5, None, None, 7, 9, None, None, 10, None, None]
Test 4 passed.


## 用循环实现deserialize函数

1. 序列的第一个数字是root，root后面跟着的一串数字（直到遇到第一个`None`之前）都属于左子树。

2. 因为是根据前序遍历重构二叉树，永远都是左子树先被填满，所以在没有特殊说想要`add_right`的情况下，都加`left child`。每次加入新的`left child`都把之前的node存入一个列表`parent`里面。这样想要回到上一个parent的时候，只需要把它`pop`出来就好了。

3. 一个数字后紧跟着两个`None`，则说明这个数字表示的是`leaf node`。只跟着一个`None`，说明这个数字表示的`node`没有右子树。两种情况，都需要回到`parent node`。  
   - ⚠️ 碰到连续两个`None`只回去上一个parent（而不是上两个parent），碰到单独的一个`None`也只回去一次。
   
4. 遇到了`None`则表示当前没有`left child`可以添加了，应当回到上一个parent，然后下一个`node`必然是`right child`。添加`right child`时，不需要保存parent，不然会导致parent的保存重复。而且每次添加完`right child`，都应该回到parent的parent，所以这里不需要再添加parent到列表里面。



In [4]:
def serialize(root):

    output = []
    if root:
        output.append(root.val)
        output = output + serialize(root.left)
        output = output + serialize(root.right)
    else:
        output.append(None)

    return output


def deserialize(s):
    
    if s is None or s == []:
        return None
    try:
        root = TreeNode(s[0])
        s = s[1:]
    except:
        return None

    parent = []
    node = root
    i = 0
    add_right = False
    while i < len(s):

        if s[i] is None:
            if i+2 >= len(s):
                # we've reach the end, the tree-reconstruction is completed
                return root
            
            node = parent.pop()
            add_right = True
            if s[i+1] == None:
                # we've reached a leaf node:
                i += 2
                continue

        else:
            if add_right:
                node.right = TreeNode(s[i])
                node = node.right
                add_right = False
            else:
                node.left = TreeNode(s[i])
                parent.append(node)
                node = node.left
        i += 1

    return root

### Test

In [6]:
root1 = TreeNode(10)
root1.left = TreeNode(6)
root1.right = TreeNode(14)

root1.left.left = TreeNode(4)
root1.left.right = TreeNode(8)

root1.right.left = TreeNode(12)
root1.right.right = TreeNode(16)

print(serialize(root1))
assert serialize(root1) == serialize( deserialize(serialize(root1)) )
print("Test 1 passed.")

[10, 6, 4, None, None, 8, None, None, 14, 12, None, None, 16, None, None]
Test 1 passed.


In [8]:
root2 = TreeNode(1)
root2.left = TreeNode(2)
root2.left.left = TreeNode(4)

root2.right = TreeNode(3)
root2.right.left = TreeNode(5)
root2.right.right = TreeNode(6)

print(serialize(root2))
assert serialize(root2) == serialize( deserialize(serialize(root2)) ) 
print("Test 2 passed.")

[1, 2, 4, None, None, None, 3, 5, None, None, 6, None, None]
Test 2 passed.


In [9]:
root3 = TreeNode(5)
root3.left = TreeNode(4)
root3.right = TreeNode(3)
root3.right.left = TreeNode(2)
# root3.right.right.right = TreeNode(1)

print(serialize(root3))
assert serialize(root3) == serialize( deserialize(serialize(root3)) ) 
print("Test 3 passed.")

[5, 4, None, None, 3, 2, None, None, None]
Test 3 passed.


In [10]:
root4 = TreeNode(8)
root4.left = TreeNode(6)
root4.right = TreeNode(7)

root4.left.left = TreeNode(10)
root4.left.right = TreeNode(5)

root4.right.left = TreeNode(9)
root4.right.right = TreeNode(10)

print(serialize(root4))
assert serialize(root4) == serialize( deserialize(serialize(root4)) ) 
print("Test 4 passed.")

[8, 6, 10, None, None, 5, None, None, 7, 9, None, None, 10, None, None]
Test 4 passed.
